In [1]:
import os

In [2]:
os.chdir('../')

In [3]:
%pwd

'/Users/apple/Documents/Personal/Computer Vision and Deep Learning/pc-parts'

In [58]:
# Type
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
  root_dir: Path
  data_dir: Path
  trained_model_path: Path
  epochs: int
  learning_rate: float
  batch_size: int
  data_name: str


In [59]:
# Configuration manager
from pcpartsclassifier.utils.common import read_yaml, create_directories
from pcpartsclassifier.constants import PARAMS_FILE_PATH, CONFIG_FILE_PATH

class ConfigurationManager:
  def __init__(self,
               config = CONFIG_FILE_PATH,
               params = PARAMS_FILE_PATH) -> None:
    self.config = read_yaml(config)
    self.params = read_yaml(params)
    create_directories([self.config.artifacts_root])
  
  def get_training_config(self) -> TrainingConfig:
    config = self.config.training
    training_config = TrainingConfig(
      root_dir = config.root_dir,
      trained_model_path = config.trained_model_path,
      data_dir= self.config.data_ingestion.root_dir,
      epochs = self.params.EPOCHS,
      learning_rate = self.params.LEARNING_RATE,
      batch_size=self.params.BATCH_SIZE,
      data_name=self.params.DATA_NAME
    )
    return training_config

In [66]:
# Training
import torch
import torch.nn as nn
from torch.utils.data import random_split, DataLoader
from torchvision import datasets, models
from pcpartsclassifier import logger

class Training:
  def __init__(self, config: TrainingConfig, model: nn.Module) -> None:
    self.config = config
    self.weights = models.EfficientNet_B0_Weights.DEFAULT
    self.transform = self.weights.transforms()
    self.data_dir = os.path.join(self.config.data_dir, self.config.data_name)
    self.model = model
    
  
  def data_preparation(self):
    dataset = datasets.ImageFolder(root=self.data_dir,
                                   transform=self.transform)
    train_set, test_set = random_split(dataset=dataset,
                                       lengths=[0.7, 0.3])
    self.train_loader = DataLoader(dataset=train_set,
                              batch_size=self.config.batch_size,
                              shuffle=True)
    self.test_loader = DataLoader(dataset=test_set,
                            batch_size=self.config.batch_size,
                            shuffle=False)
    self.classes = dataset.classes
  
  def train_loop(self, optimizer, criterion: nn.Module):
    train_loss = 0
    for _, (images, labels) in enumerate(self.train_loader):
      self.model.train()
      # forward propagation and loss
      logits = self.model(images)
      loss = criterion(logits, labels)
      train_loss += loss.item()
      # zero_grad
      optimizer.zero_grad()

      # backward propagation
      loss.backward()

      # step
      optimizer.step()
    return train_loss

  def test_loop(self, criterion: nn.Module):
    test_loss = 0
    self.model.eval()
    with torch.inference_mode():
      # forward pass
      for _, (images, labels) in enumerate(self.test_loader):
        logits = self.model(images)
        loss = criterion(logits, labels)
        test_loss += loss.item()
    return test_loss

  def train(self, model:nn.Module):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(params=model.parameters(),
                                 lr=self.config.learning_rate)
    self.data_preparation()
    for epoch in range(self.config.epochs):
      train_loss = self.train_loop(model=model,
                      optimizer=optimizer,
                      criterion=criterion)
      test_loss /= len(self.train_loader)

      logger.info(f'Epoch: {epoch} | Loss: {train_loss:.3f}')
      test_loss = self.test_loop(model=model,
                     criterion=criterion)
      test_loss /= len(self.test_loader)
      logger.info(f'Epoch: {epoch} | Loss: {test_loss:.3f}')
  
  def save_model(self):
    create_directories([self.config.root_dir])
    torch.save(self.model.state_dict(), self.config.trained_model_path)
      


In [65]:
# Pipeline
try:
  config = ConfigurationManager()
  training_config = config.get_training_config()
  training = Training()
  training.train()
except Exception as e:
  raise e
  

[2024-03-30 16:09:46,422 : INFO : common : yaml file: config/config.yaml loaded successfull]
[2024-03-30 16:09:46,473 : INFO : common : yaml file: params.yaml loaded successfull]
[2024-03-30 16:09:46,475 : INFO : common : created directory at artifacts]


TypeError: Training.__init__() missing 1 required positional argument: 'model'